In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install datasets evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=43b6cb86a07945a22fe7bd9fc82bd3b6cd8261c31970e08b28ec491ad18e6342
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is

In [3]:
import pandas as pd
import numpy as np
import torch
import re, unicodedata
from datasets import Dataset
from transformers import (
  AutoTokenizer,
  AutoModelForSeq2SeqLM,
  Seq2SeqTrainingArguments,
  Seq2SeqTrainer,
  DataCollatorForSeq2Seq
)
from rouge_score import rouge_scorer


df_train = pd.read_csv('/kaggle/input/ilsum-hindi/hindi_ilsum_2024_train.csv')
df_val = pd.read_csv('/kaggle/input/ilsum-hindi/hindi_ilsum_2024_val.csv')


# Text preprocessing
def normalize_text(text):
  text = str(text).strip()
  text = unicodedata.normalize("NFKC", text)
  text = re.sub(r'(.)\1{2,}', r'\1\1', text)
  return text

def clean_mixed_hindi_english(text):
  text = normalize_text(text)
  text = re.sub(r"http\S+|www\S+", "[URL]", text)
  text = re.sub(r"@\w+", "[USER]", text)
  text = re.sub(r"[^\w\s\u0900-\u097F]", " ", text)
  text = re.sub(r"\s+", " ", text).strip()
  return text

df_train["Article"] = df_train["Article"].apply(clean_mixed_hindi_english)
df_train["Summary"] = df_train["Summary"].apply(clean_mixed_hindi_english)
df_val["Article"] = df_val["Article"].apply(clean_mixed_hindi_english)
df_val["Summary"] = df_val["Summary"].apply(clean_mixed_hindi_english)

train_dataset = Dataset.from_pandas(df_train[['Article', 'Summary']])
val_dataset = Dataset.from_pandas(df_val[['Article', 'Summary']])


# Initializing model
model_name = "ai4bharat/IndicBART"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


# Preprocess function for tokenization

def preprocess_function(examples):
  inputs = tokenizer(
    examples["Article"],
    max_length=512,
    truncation=True,
    padding="max_length"
  )

  labels = tokenizer(
    examples["Summary"],
    max_length=128,
    truncation=True,
    padding="max_length"
  )

  labels["input_ids"] = [
    [(token if token != tokenizer.pad_token_id else -100) for token in label]
    for label in labels["input_ids"]
  ]

  inputs["labels"] = labels["input_ids"]
  return inputs


# Apply preprocessing
tokenized_train = train_dataset.map(
  preprocess_function,
  batched=True,
  remove_columns=train_dataset.column_names
)
tokenized_val = val_dataset.map(
  preprocess_function,
  batched=True,
  remove_columns=val_dataset.column_names
)


# Data collator
data_collator = DataCollatorForSeq2Seq(
  tokenizer=tokenizer,
  model=model,
  padding=True
)


# Helper function for computing ROUGE metrics
def compute_metrics(pred):
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

  predictions = tokenizer.batch_decode(
      pred.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
  )

  # Replace -100 in the labels as we can't decode them
  labels = np.where(pred.label_ids != -100, pred.label_ids, tokenizer.pad_token_id)
  references = tokenizer.batch_decode(
      labels, skip_special_tokens=True, clean_up_tokenization_spaces=True
  )

  # Compute ROUGE scores
  rouge_scores = {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}
  for pred, ref in zip(predictions, references):
    scores = scorer.score(ref, pred)
    rouge_scores['rouge1'] += scores['rouge1'].fmeasure
    rouge_scores['rouge2'] += scores['rouge2'].fmeasure
    rouge_scores['rougeL'] += scores['rougeL'].fmeasure

  # Average scores
  num_samples = len(predictions)
  for key in rouge_scores:
      rouge_scores[key] /= num_samples

  return rouge_scores



# Training arguments
training_args = Seq2SeqTrainingArguments(
  output_dir="/kaggle/working/",
  eval_strategy="epoch",
  learning_rate=3e-5,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  weight_decay=0.01,
  save_total_limit=3,
  num_train_epochs=6,
  predict_with_generate=True,
  report_to="tensorboard",
  save_strategy="epoch",
  load_best_model_at_end=True,
  metric_for_best_model="rouge1",
)



# Initialize the trainer
trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_train,
  eval_dataset=tokenized_val,
  tokenizer=tokenizer,
  data_collator=data_collator,
  compute_metrics=compute_metrics
)



# Training
print("Starting training...")
trainer.train()
print("Training completed!")


# Save model and tokenizer
model.save_pretrained("/kaggle/working/indicbart-summarizer")
tokenizer.save_pretrained("/kaggle/working/indicbart-summarizer")



# Inference function for using the model
def generate_summary(article_text, model, tokenizer, max_length=128):
  inputs = tokenizer(
    article_text,
    max_length=128,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
  )

  # Move to GPU if available
  if torch.cuda.is_available():
    inputs = {k: v.to("cuda") for k, v in inputs.items()}
    model = model.to("cuda")

  # Generate summary
  summary_ids = model.generate(
    inputs["input_ids"],
    num_beams=4,
    min_length=30,
    max_length=max_length,
    early_stopping=True,
    no_repeat_ngram_size=3
  )

  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return summary


# Example usage
sample_article = df_val.iloc[0]["Article"]
model_path = "/kaggle/working/indicbart-summarizer"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
summary = generate_summary(sample_article, model, tokenizer)
print(f"Generated Summary: {summary}")


2025-09-12 21:25:55.289501: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757712355.485014      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757712355.549313      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

...

Starting training...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
1,2.265300,1.875014,0.501543,0.200981,0.499694
2,1.925100,1.723624,0.505581,0.205679,0.503958
3,1.774300,1.661521,0.505942,0.207001,0.504329
4,1.657200,1.590083,0.504637,0.209316,0.502883
5,1.619100,1.573427,0.504387,0.208205,0.502673
6,1.584000,1.556304,0.506251,0.210574,0.504537


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

Training completed!
Generated Summary: MeToo कैंपेन के जरिए पत्रकारिता जगत का एक और बड़ा नाम सामने आया है महिला पत्रकार निष्ठा जैन ने फेसबुक के जरिए वरिष्ठ पत्रकार विनोद दुआ पर यौन उत्पीड़न का आरोप लगाया है महिला ने फेसबुक पर एक लंबा चौड़ा पोस्ट लिखकर विनोद दुआ के साथ दो घटनाओं का जिक्र किया है जिनमें उनका उत्पीड़न किया गया पहली घटना का जिक्र करते हुए निष्ठा जैन लिखती हैं कि 1989 में वह इंटरव्यू के लिए विनोद दुआ से मिली थी उन्होंने अपने साथ हुई यौन उत्पीड़न की घटनाओं को सामने ला चुकी हैं एक्ट्रेस ने अपने इस पोस्ट में उन्होंने बताया कि कैसे उनके साथ उत्पीड़न की घटनाएं सामने आई हैं
